In [73]:
from pathlib import Path
import nuscenes
import pandas as pd
import numpy as np


### Prepare kitti dataset from nuscenes

In [36]:
# create soft link for kitti data inside pointrcnn
!ln -s /media/ags/DATA/CODE/kaggle/lyft-3d-object-detection/data/nusc_kitti/training/ /media/ags/DATA/CODE/kaggle/lyft-3d-object-detection/PointRCNN/data/KITTI/object/training

In [38]:
!ls data/KITTI/object/training/velodyne/ | wc -l

404


In [9]:
dataset = nuscenes.NuScenes(dataroot='../data/nuscenes/v1.0-mini/', version='v1.0-mini')

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.4 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [54]:
len(dataset.sample)

404

In [50]:
is_train = []
for sample in dataset.sample:
    scene = dataset.get('scene', sample['scene_token'])
    if scene['name'] in mini_train:
        is_train.append(1)
    else:
        is_train.append(0)

In [57]:
trainval = ['%06d' % x for x in range(404)]
train = []
val = []
for x in range(len(trainval)):
    if is_train[x]:
        train.append(trainval[x])
    else:
        val.append(trainval[x])

In [59]:
len(train), len(val)

(323, 81)

In [71]:
trainval_file = data_dir / 'ImageSets/trainval.txt'
train_file = data_dir / 'ImageSets/train.txt'
val_file = data_dir / 'ImageSets/val.txt'
with open(trainval_file, 'w') as f:
    for x in trainval:
        f.write(x + '\n')
with open(train_file, 'w') as f:
    for x in train:
        f.write(x + '\n')
with open(val_file, 'w') as f:
    for x in val:
        f.write(x + '\n')

In [72]:
len(train + val) == len(trainval)

True

### lidar file loading testing 

In [77]:
file = str(data_dir / 'object/training/velodyne/000000.bin')
lidar = np.fromfile(file)

In [81]:
lidar.reshape(-1, 4).shape

(17344, 4)

In [90]:
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud

In [130]:
#lyft_lidar = np.fromfile('../data/lyft/lyft_trainval/lidar/host-a011_lidar1_1232731615601286686.bin')
#file = Path('../data/lyft/lyft_trainval/lidar/host-a011_lidar1_1232731611201284806.bin')
file = Path('data/KITTI/object/training/velodyne/000000.bin')
lyft_lidar = np.fromfile(str(file), dtype=np.float32) # IT is extremely important that you specify dtype
# lp = LidarPointCloud.from_file(file)
lp.points.shape, lyft_lidar.shape

((4, 67409), (138752,))

In [140]:
base = '../data/nuscenes/v1.0-mini/'
for sample in dataset.sample:
    file = dataset.get('sample_data', sample['data']['LIDAR_TOP'])['filename']
    lidar = LidarPointCloud.from_file(Path(base + file))
    print(lidar.points.shape)

(4, 34688)
(4, 34720)
(4, 34720)
(4, 34688)
(4, 34752)
(4, 34784)
(4, 34720)
(4, 34720)
(4, 34688)
(4, 34688)
(4, 34688)
(4, 34720)
(4, 34688)
(4, 34752)
(4, 34752)
(4, 34784)
(4, 34784)
(4, 34752)
(4, 34688)
(4, 34688)
(4, 34688)
(4, 34720)
(4, 34688)
(4, 34720)
(4, 34752)
(4, 34720)
(4, 34720)
(4, 34720)
(4, 34752)
(4, 34720)
(4, 34720)
(4, 34720)
(4, 34720)
(4, 34752)
(4, 34720)
(4, 34688)
(4, 34688)
(4, 34720)
(4, 34720)
(4, 34752)
(4, 34688)
(4, 34752)
(4, 34720)
(4, 34784)
(4, 34720)
(4, 34688)
(4, 34688)
(4, 34720)
(4, 34720)
(4, 34720)
(4, 34688)
(4, 34688)
(4, 34688)
(4, 34720)
(4, 34688)
(4, 34720)
(4, 34720)
(4, 34784)
(4, 34784)
(4, 34752)
(4, 34752)
(4, 34752)
(4, 34720)
(4, 34688)
(4, 34688)
(4, 34688)
(4, 34752)
(4, 34720)
(4, 34720)
(4, 34688)
(4, 34688)
(4, 34752)
(4, 34784)
(4, 34688)
(4, 34720)
(4, 34752)
(4, 34720)
(4, 34752)
(4, 34720)
(4, 34400)
(4, 34688)
(4, 34720)
(4, 34720)
(4, 34688)
(4, 34688)
(4, 34720)
(4, 34688)
(4, 34752)
(4, 34752)
(4, 34752)
(4, 34720)

In [150]:
# base = '../data/nuscenes/v1.0-mini/'
base = 'data/KITTI/object/training/velodyne/'
import os
import glob
for file in glob.glob(base + '*'):
    lidar = np.fromfile(file, dtype=np.float32).reshape(-1, 4)
    print(lidar.shape)


(34784, 4)
(34688, 4)
(34720, 4)
(34688, 4)
(34720, 4)
(34752, 4)
(34752, 4)
(34752, 4)
(34720, 4)
(34688, 4)
(34720, 4)
(34720, 4)
(34752, 4)
(34752, 4)
(34784, 4)
(34688, 4)
(34752, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34752, 4)
(34752, 4)
(34720, 4)
(34720, 4)
(34688, 4)
(34752, 4)
(34688, 4)
(34752, 4)
(34720, 4)
(34400, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34784, 4)
(34720, 4)
(34720, 4)
(34784, 4)
(34688, 4)
(34688, 4)
(34688, 4)
(34784, 4)
(34720, 4)
(34752, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34656, 4)
(34752, 4)
(34720, 4)
(34752, 4)
(34720, 4)
(34688, 4)
(34720, 4)
(34688, 4)
(34720, 4)
(34720, 4)
(34784, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34688, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34688, 4)
(34688, 4)
(34752, 4)
(34720, 4)
(34752, 4)
(34688, 4)
(34784, 4)
(34720, 4)
(34656, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34720, 4)
(34688, 4)
(34720, 4)
(34720, 4)
(34720, 4)

In [ ]:
# lidar points are ~ (4, 34700)

In [151]:
base = '../data/KITTI/object/training/velodyne/'
import os
import glob
for file in glob.glob(base + '*'):
    lidar = np.fromfile(file, dtype=np.float32).reshape(-1, 4)
    print(lidar.shape)


(112321, 4)
(115167, 4)
(117190, 4)
(119065, 4)
(111220, 4)
(115113, 4)
(123807, 4)
(108049, 4)
(122652, 4)
(126372, 4)
(114367, 4)
(122473, 4)
(113238, 4)
(115459, 4)
(115137, 4)
(109235, 4)
(124773, 4)
(125622, 4)
(114805, 4)
(120272, 4)
(121157, 4)
(123916, 4)
(119282, 4)
(113682, 4)
(106713, 4)
(118330, 4)
(113183, 4)
(124200, 4)
(119518, 4)
(126161, 4)
(118477, 4)
(124579, 4)
(120377, 4)
(118328, 4)
(123205, 4)
(113249, 4)
(119088, 4)
(122548, 4)
(117744, 4)
(119209, 4)
(116802, 4)
(123313, 4)
(118610, 4)
(118329, 4)
(123241, 4)
(117040, 4)
(121199, 4)
(119293, 4)
(118527, 4)
(117134, 4)
(123009, 4)
(110103, 4)
(123208, 4)
(121764, 4)
(119053, 4)
(115875, 4)
(122259, 4)
(120972, 4)
(115114, 4)
(123843, 4)
(122593, 4)
(122845, 4)
(121930, 4)
(122441, 4)
(122910, 4)
(124449, 4)
(120269, 4)
(103893, 4)
(114661, 4)
(114380, 4)
(116332, 4)
(123591, 4)
(119167, 4)
(119238, 4)
(116335, 4)
(122769, 4)
(115153, 4)
(109263, 4)
(121555, 4)
(110394, 4)
(121336, 4)
(122705, 4)
(110030, 4)
(125

(117094, 4)
(125045, 4)
(123593, 4)
(121977, 4)
(117810, 4)
(119301, 4)
(117019, 4)
(119674, 4)
(121270, 4)
(123521, 4)
(115649, 4)
(126414, 4)
(113294, 4)
(115315, 4)
(124208, 4)
(120214, 4)
(118679, 4)
(115822, 4)
(119199, 4)
(122058, 4)
(117708, 4)
(119616, 4)
(111572, 4)
(103629, 4)
(121117, 4)
(126229, 4)
(112614, 4)
(123394, 4)
(110024, 4)
(124619, 4)
(122590, 4)
(125837, 4)
(126282, 4)
(103670, 4)
(114683, 4)
(120962, 4)
(115023, 4)
(117497, 4)
(124289, 4)
(115055, 4)
(123344, 4)
(125204, 4)
(114272, 4)
(122307, 4)
(124793, 4)
(121251, 4)
(124066, 4)
(123451, 4)
(116044, 4)
(123640, 4)
(123365, 4)
(120504, 4)
(125834, 4)
(121725, 4)
(120456, 4)
(123260, 4)
(108008, 4)
(114174, 4)
(124020, 4)
(127172, 4)
(120427, 4)
(113349, 4)
(115069, 4)
(114298, 4)
(112820, 4)
(118090, 4)
(116360, 4)
(124313, 4)
(101770, 4)
(119262, 4)
(113443, 4)
(126719, 4)
(114367, 4)
(116277, 4)
(120219, 4)
(126246, 4)
(122087, 4)
(116486, 4)
(119356, 4)
(120211, 4)
(113566, 4)
(118889, 4)
(118543, 4)
(114

(122973, 4)
(114338, 4)
(122525, 4)
(121653, 4)
(115661, 4)
(119238, 4)
(122675, 4)
(100477, 4)
(120236, 4)
(114980, 4)
(118270, 4)
(117547, 4)
(115181, 4)
(117905, 4)
(123493, 4)
(123088, 4)
(114941, 4)
(119338, 4)
(118663, 4)
(119599, 4)
(121555, 4)
(101787, 4)
(122267, 4)
(125527, 4)
(126670, 4)
(125171, 4)
(119682, 4)
(121524, 4)
(123264, 4)
(122781, 4)
(123063, 4)
(95289, 4)
(112798, 4)
(120713, 4)
(123669, 4)
(118938, 4)
(121116, 4)
(121093, 4)
(126177, 4)
(113257, 4)
(121667, 4)
(122261, 4)
(121604, 4)
(123444, 4)
(116268, 4)
(121112, 4)
(124772, 4)
(122373, 4)
(125463, 4)
(110899, 4)
(115805, 4)
(108808, 4)
(121229, 4)
(114996, 4)
(126458, 4)
(118457, 4)
(118746, 4)
(123733, 4)
(121577, 4)
(123457, 4)
(122659, 4)
(122149, 4)
(118451, 4)
(124121, 4)
(103471, 4)
(101653, 4)
(114177, 4)
(114347, 4)
(123145, 4)
(108348, 4)
(121242, 4)
(126766, 4)
(123513, 4)
(121758, 4)
(117652, 4)
(124885, 4)
(122324, 4)
(124232, 4)
(120039, 4)
(124211, 4)
(120768, 4)
(122265, 4)
(118207, 4)
(1201

(105742, 4)
(116126, 4)
(121531, 4)
(117073, 4)
(115079, 4)
(115190, 4)
(119414, 4)
(122321, 4)
(122399, 4)
(115892, 4)
(123611, 4)
(114163, 4)
(106696, 4)
(114560, 4)
(121440, 4)
(122218, 4)
(120663, 4)
(114249, 4)
(123687, 4)
(121866, 4)
(122635, 4)
(115399, 4)
(124540, 4)
(118007, 4)
(122516, 4)
(117525, 4)
(122621, 4)
(113869, 4)
(119684, 4)
(103857, 4)
(120612, 4)
(113832, 4)
(126875, 4)
(123923, 4)
(128091, 4)
(116847, 4)
(122888, 4)
(112093, 4)
(115067, 4)
(116663, 4)
(121759, 4)
(124761, 4)
(122798, 4)
(123936, 4)
(119813, 4)
(119468, 4)
(125826, 4)
(122637, 4)
(123191, 4)
(119232, 4)
(123193, 4)
(123298, 4)
(118789, 4)
(123545, 4)
(123188, 4)
(125358, 4)
(117874, 4)
(120407, 4)
(126265, 4)
(124267, 4)
(120988, 4)
(116091, 4)
(121784, 4)
(123104, 4)
(118734, 4)
(123962, 4)
(121700, 4)
(121446, 4)
(123033, 4)
(121303, 4)
(113575, 4)
(119895, 4)
(119676, 4)
(124202, 4)
(120002, 4)
(123852, 4)
(118805, 4)
(120097, 4)
(116489, 4)
(121324, 4)
(115357, 4)
(117889, 4)
(107859, 4)
(116

(122087, 4)
(121235, 4)
(122118, 4)
(115211, 4)
(122606, 4)
(121023, 4)
(123884, 4)
(124202, 4)
(116869, 4)
(124454, 4)
(123474, 4)
(118593, 4)
(121497, 4)
(124055, 4)
(118874, 4)
(124606, 4)
(115760, 4)
(122252, 4)
(121687, 4)
(120829, 4)
(108650, 4)
(115254, 4)
(126536, 4)
(118817, 4)
(114305, 4)
(118070, 4)
(120599, 4)
(123456, 4)
(118025, 4)
(122417, 4)
(118749, 4)
(119812, 4)
(114801, 4)
(120151, 4)
(119748, 4)
(118514, 4)
(115227, 4)
(118733, 4)
(125665, 4)
(115566, 4)
(117377, 4)
(128312, 4)
(123644, 4)
(108473, 4)
(105701, 4)
(121656, 4)
(122133, 4)
(118385, 4)
(123892, 4)
(119867, 4)
(116823, 4)
(116538, 4)
(123618, 4)
(123125, 4)
(115071, 4)
(119160, 4)
(109569, 4)
(113060, 4)
(120240, 4)
(123923, 4)
(120647, 4)
(120550, 4)
(116917, 4)
(117733, 4)
(119093, 4)
(121468, 4)
(113405, 4)
(124975, 4)
(118243, 4)
(119103, 4)
(120067, 4)
(123904, 4)
(119761, 4)
(124168, 4)
(124152, 4)
(124056, 4)
(110195, 4)
(119362, 4)
(121119, 4)
(122063, 4)
(118307, 4)
(124473, 4)
(117886, 4)
(123

...

(123069, 4)
(110337, 4)
(126003, 4)
(122791, 4)
(125062, 4)
(118460, 4)
(124350, 4)
(126462, 4)
(123448, 4)
(114503, 4)
(119279, 4)
(121210, 4)
(125338, 4)
(121713, 4)
(111396, 4)
(117539, 4)
(120008, 4)
(123459, 4)
(124517, 4)
(113974, 4)
(125262, 4)
(121344, 4)
(126657, 4)
(124435, 4)
(126097, 4)
(107561, 4)
(123965, 4)
(119734, 4)
(114262, 4)
(118577, 4)
(117408, 4)
(125156, 4)
(120758, 4)
(125044, 4)
(119645, 4)
(109489, 4)
(125335, 4)
(125725, 4)
(109842, 4)
(122159, 4)
(126511, 4)
(123637, 4)
(116946, 4)
(120382, 4)
(115830, 4)
(121380, 4)
(118862, 4)
(118940, 4)
(120015, 4)
(124347, 4)
(125770, 4)
(122419, 4)
(122644, 4)
(122985, 4)
(116488, 4)
(124882, 4)
(123677, 4)
(122948, 4)
(115178, 4)
(120690, 4)
(120067, 4)
(111980, 4)
(110653, 4)
(124132, 4)
(121610, 4)
(123497, 4)
(118137, 4)
(115114, 4)
(117142, 4)
(116810, 4)
(113739, 4)
(122058, 4)
(123219, 4)
(119306, 4)
(123440, 4)
(118346, 4)
(124762, 4)
(117744, 4)
(123221, 4)
(117701, 4)
(118675, 4)
(125815, 4)
(120229, 4)
(125

(116055, 4)
(115141, 4)
(122809, 4)
(121422, 4)
(121338, 4)
(124211, 4)
(118702, 4)
(123847, 4)
(125386, 4)
(118864, 4)
(118566, 4)
(115141, 4)
(124644, 4)
(121883, 4)
(121890, 4)
(114080, 4)
(122105, 4)
(123707, 4)
(115095, 4)
(115179, 4)
(115285, 4)
(120590, 4)
(122769, 4)
(109787, 4)
(104976, 4)
(127270, 4)
(118718, 4)
(122766, 4)
(120886, 4)
(125073, 4)
(121942, 4)
(115129, 4)
(119786, 4)
(123570, 4)
(124913, 4)
(123587, 4)
(117421, 4)
(114278, 4)
(122199, 4)
(116508, 4)
(116866, 4)
(119776, 4)
(118859, 4)
(114295, 4)
(123743, 4)
(114166, 4)
(123168, 4)
(122212, 4)
(122214, 4)
(110402, 4)
(122262, 4)
(122478, 4)
(114327, 4)
(118213, 4)
(115346, 4)
(114750, 4)
(122846, 4)
(124559, 4)
(112865, 4)
(125866, 4)
(122730, 4)
(110096, 4)
(122512, 4)
(125216, 4)
(124815, 4)
(115757, 4)
(119648, 4)
(121828, 4)
(118405, 4)
(115853, 4)
(124050, 4)
(118486, 4)
(123582, 4)
(114845, 4)
(125049, 4)
(115281, 4)
(118617, 4)
(121984, 4)
(117817, 4)
(118384, 4)
(123599, 4)
(118959, 4)
(126829, 4)
(124

(120910, 4)
(120854, 4)
(118510, 4)
(125048, 4)
(104337, 4)
(115122, 4)
(126394, 4)
(115032, 4)
(108943, 4)
(123618, 4)
(123492, 4)
(125972, 4)
(114849, 4)
(123820, 4)
(111423, 4)
(123564, 4)
(115211, 4)
(114380, 4)
(107943, 4)
(122939, 4)
(122709, 4)
(121100, 4)
(124748, 4)
(121965, 4)
(116996, 4)
(109353, 4)
(120107, 4)
(123199, 4)
(120023, 4)
(123419, 4)
(121494, 4)
(110614, 4)
(120092, 4)
(125013, 4)
(120741, 4)
(125840, 4)
(112679, 4)
(123496, 4)
(124521, 4)
(124402, 4)
(121662, 4)
(122688, 4)
(118017, 4)
(125499, 4)
(118849, 4)
(113682, 4)
(124183, 4)
(114785, 4)
(118446, 4)
(122580, 4)
(109107, 4)
(121369, 4)
(125098, 4)
(119507, 4)
(115039, 4)
(121252, 4)
(115123, 4)
(126626, 4)
(123658, 4)
(118522, 4)
(120804, 4)
(115092, 4)
(116523, 4)
(117173, 4)
(119744, 4)
(125578, 4)
(111155, 4)
(126391, 4)
(105250, 4)
(122479, 4)
(123266, 4)
(117178, 4)
(113220, 4)
(117139, 4)
(109690, 4)
(118837, 4)
(112139, 4)
(115225, 4)
(117207, 4)
(122159, 4)
(122881, 4)
(122145, 4)
(121388, 4)
(121

(122083, 4)
(125764, 4)
(122495, 4)
(122317, 4)
(115236, 4)
(122458, 4)
(126575, 4)
(115698, 4)
(115300, 4)
(122939, 4)
(126702, 4)
(119394, 4)
(122264, 4)
(121782, 4)
(119709, 4)
(115394, 4)
(123679, 4)
(116099, 4)
(114338, 4)
(120889, 4)
(123625, 4)
(120879, 4)
(109367, 4)
(112530, 4)
(127176, 4)
(117260, 4)
(109688, 4)
(109027, 4)
(103976, 4)
(117362, 4)
(122185, 4)
(118062, 4)
(123682, 4)
(122335, 4)
(115203, 4)
(107033, 4)
(124646, 4)
(122485, 4)
(122331, 4)
(112806, 4)
(128467, 4)
(108339, 4)
(125715, 4)
(123370, 4)
(117344, 4)
(111377, 4)
(120338, 4)
(110578, 4)
(115338, 4)
(120219, 4)
(126610, 4)
(122271, 4)
(123995, 4)
(114957, 4)
(116585, 4)
(124033, 4)
(119315, 4)
(119199, 4)
(115116, 4)
(90398, 4)
(122642, 4)
(104395, 4)
(119540, 4)
(115796, 4)
(119674, 4)
(120640, 4)
(120344, 4)
(117547, 4)
(118729, 4)
(109595, 4)
(109526, 4)
(121547, 4)
(125047, 4)
(119639, 4)
(115481, 4)
(120744, 4)
(120324, 4)
(125009, 4)
(123111, 4)
(124935, 4)
(124599, 4)
(113998, 4)
(114320, 4)
(1200

(123583, 4)
(125885, 4)
(126221, 4)
(123575, 4)
(115889, 4)
(123524, 4)
(126262, 4)
(123553, 4)
(122116, 4)
(123343, 4)
(122672, 4)
(113959, 4)
(123063, 4)
(124977, 4)
(119744, 4)
(114694, 4)
(122011, 4)
(122942, 4)
(115040, 4)
(120657, 4)
(123744, 4)
(119931, 4)
(120122, 4)
(114205, 4)
(119185, 4)
(108204, 4)
(119104, 4)
(116902, 4)
(117242, 4)
(124260, 4)
(119942, 4)
(124909, 4)
(115020, 4)
(102390, 4)
(124821, 4)
(112387, 4)
(126275, 4)
(124294, 4)
(122715, 4)
(126316, 4)
(115125, 4)
(115035, 4)
(123789, 4)
(123390, 4)
(114451, 4)
(123910, 4)
(119036, 4)
(123808, 4)
(117648, 4)
(120356, 4)
(116549, 4)
(116313, 4)
(125418, 4)
(122594, 4)
(114589, 4)
(109139, 4)
(120028, 4)
(109740, 4)
(124690, 4)
(122244, 4)
(111337, 4)
(122744, 4)
(116092, 4)
(122216, 4)
(114210, 4)
(124203, 4)
(123592, 4)
(119552, 4)
(118037, 4)
(123431, 4)
(122525, 4)
(114393, 4)
(123843, 4)
(123339, 4)
(119065, 4)
(123097, 4)
(111632, 4)
(121240, 4)
(112974, 4)
(124973, 4)
(125830, 4)
(117647, 4)
(113127, 4)
(127

In [133]:
lyft_lidar.reshape((-1, 4)).shape

(34688, 4)

In [132]:
lyft_lidar.reshape(-1, 5).shape

ValueError: cannot reshape array of size 138752 into shape (5)

In [96]:
file_name = Path('../data/lyft/lyft_trainval/lidar/host-a011_lidar1_1232731611201284806.bin')
ll = LidarPointCloud.from_file(file_name)

In [97]:
ll.points.shape

(4, 67409)

In [111]:
scan = np.fromfile(str(file), dtype=np.float32)
print(scan.shape)
points = scan.reshape((-1, 5))#[:, : 4]
points.shape

(337045,)


(67409, 5)

### testing

In [126]:
dataset.category

[{'token': '1fa93b757fc74fb197cdd60001ad8abf',
  'name': 'human.pedestrian.adult',
  'description': 'Adult subcategory.'},
 {'token': 'b1c6de4c57f14a5383d9f963fbdcb5cb',
  'name': 'human.pedestrian.child',
  'description': 'Child subcategory.'},
 {'token': 'b2d7c6c701254928a9e4d6aac9446d79',
  'name': 'human.pedestrian.wheelchair',
  'description': 'Wheelchairs. If a person is in the wheelchair, include in the annotation.'},
 {'token': '6a5888777ca14867a8aee3fe539b56c4',
  'name': 'human.pedestrian.stroller',
  'description': 'Strollers. If a person is in the stroller, include in the annotation.'},
 {'token': '403fede16c88426885dd73366f16c34a',
  'name': 'human.pedestrian.personal_mobility',
  'description': 'A small electric or self-propelled vehicle, e.g. skateboard, segway, or scooters, on which the person typically travels in a upright position. Driver and (if applicable) rider should be included in the bounding box along with the vehicle.'},
 {'token': 'bb867e2064014279863c71a29b1